<img src="https://raw.githubusercontent.com/DonAurelio/open-datacube-bac-training/main/docs/banner.png" alt="Deparatemento de Ingeniería de Sistemas y Computación, Universidad de los Andes">

#  Análisis espacial - Aplicación de algoritmos para el análisis de coberturas

**Introducción**

La teledetección es el proceso de **detectar** y **monitorear** las características físicas, químicas y biológicas de la cobertura terrestre. Estas caracteristicas pueden ser estudiadas mediante el análisis de la radiación reflejada y emitida a distancia por los diferentes tipos de coberturas que reposan sobre la superficie terrestre. 

En los notebook 3 y 4 se utilizó la estrategia de **visualización** en `verdadero color` y en `falso color`, que son un primer paso de análisis y donde el análisis es realizado en forma visual por el analista.

Sin embargo hay otras muchas posibilidades de análisis que involucran operaciones matemáticas con las bandas de la imagem. En este notebook se muestran algunos algoritmos (índices de vegetación) empleados de forma recurrente en la literatura para el estudio de cultivos. Así mismo se muestran algoritmos que permiten mitigar el efecto de las nubes que producen valores inválidos para el análisis de un cultivo. 

Luego de realizar un análisis de las cobertura terrestre con estos índices, se exportan los resultados de los análisis en formatos conocidos para su exploración en herramientas GIS como ArcGIS o Q-GIS

**Objetivo**

Familiarizarse con el proceso de análisis de imágenes satelitales usando el cubo de datos.

**Precondiciones - Actividad previa**

1. Tener dos imágenes en el cubo - Ver notebooks 1 y 2
2. Haber realizado las actividades de los notebooks 3 y 4

**Contenido**

1. Importar librerías
2. Definición del área de estudio y consulta de las imágenes
3. Cálculo de índices de vegetación
4. Guardar resultados de análisis en formato netcdf
5. Guardar resultados de análisis en formato geotiff

## 1. Importar librerías
___
En esta sección se importan las librerías cuya funicionalidades particulares son requeridas.

In [ ]:
# las funcionalidades del open data cube son accedidas 
# por medio de la librería datacube
import datacube

# Manipulación de datasets
import xarray as xr

# Manipulación de datos raster
import rasterio

# Librería usada para la carga de polígonos
import geopandas as gpd

# Librería usada para visualización de datos
import matplotlib as mpl
import matplotlib.pyplot as plt

# Desactiva los warnings en el notebook
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## 2. Definición del área de estudio y consulta de las imágenes
___
Para usar el cubo de datos y las imágenes que contiene, lo primero que hay que hacer es contarle al cubo cuál es el área en la que se quiere trabajar, expresada como un rectángulo que define sus límites.

En este notebook, esta área de estudio se especifica a partir de un punto.

Los coordenadas del punto a seleccionar pueden ser obtenidas a través de herramientas GIS como Google Maps. Este punto debe estar comprendido en el área que desea estudiar. El punto definido se emplea para la generación de un cuadrado que finalmente se usa para consultar el área de estudio. La variable `buffer` permite ampliar o disminuir las dimensiones de dicho cuadrado. Lo anterior es equivalente a disminuir o ampliar el área de estudio a consultar en el open data cube.

<img src="https://raw.githubusercontent.com/DonAurelio/open-datacube-bac-training/main/docs/latlong_buffer.png" alt="Definición área de estudio" width="20%">

In [ ]:
# Definición de las coordenadas del punto central del área de estudio
central_lat = 5.547964746532565
central_lon =  -72.9284962779124

# Aumento del aŕea del cuadrado para "EPSG:4326" (WGS84 - Unidades en grados)
#  0.1 grados que corresponden a 11.1 kilómetros alrededor del punto de interés
buffer = 0.1

# Cálculo del cuadro delimitador (bounding box) para el área de estudio
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

print(f'    latitude={study_area_lat},')
print(f'    longitude={study_area_lon},')

**Ahora recuperamos las imágenes que tenga el cubo y que contienen el área de interés**

In [ ]:
dc = datacube.Datacube(app="MOOC GEO")

# Especificación de los parámetros de búsqueda
dataset = dc.load(
    product="s2_sen2cor_ard_granule_EO3",
    latitude=(5.447964746532565, 5.647964746532565),
    longitude=(-73.0284962779124, -72.82849627791241),
    time=('2021-01-01', '2021-02-01'),
    measurements=["red","blue","green","nir","swir1","swir2","scl"],
    crs="EPSG:4326",
    output_crs="EPSG:4326",
    resolution=(-0.00008983111,0.00008971023)
)

# Ver el resultado de la consulta, en el formato del cubo
dataset

Al igual que en los notebooks anteriores, también queremos ver las imágenes recuperadas

In [ ]:
rgb = dataset[["red","green","blue"]].to_array(dim='color')
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension
img = rgb.plot.imshow(col='time',col_wrap=4,add_colorbar=False,vmin=0,vmax=1200)

Recordemos que también podemos visualizar una sola imagen, de alguno de los periodos de tiempo recuperados.

En el notebook 3, lo hicimos escogiendo la primera (y única) imagen que teníamos, con la instrucción:

`img = rgb.plot.imshow(add_colorbar=False,vmin=0,vmax=1500, size=10, aspect=1)`

En este notebook, mostramos otra alternativa, mediante la utilización de la variable `time_index`. Y también mostramos otra alternativa para determinar el tamaño de la imagen resultado, mediante el parámetro `figsize`.

**NOTA:** Entre más grande es la imagen, más tiempo de procesamiento se requiere para su visualización.

In [ ]:
time_index = 0

rgb = dataset[["red","green","blue"]].isel(time=time_index).to_array(dim='color')
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension
img = rgb.plot.imshow(add_colorbar=True, vmin=0,vmax=1200,figsize=(5,5))

## 3. Cálculo de índices de vegetación
___
Los índices de vegetación son el resultado de operar aritméticamente los componentes espectrales (bandas) de una imagen satelital, con el objetivo de realzar las propiedades fenológicas de la vegetación. 

Ejemplos  de  índices  de vegetación más comunmente empleados son: 
- **Normalized Difference Vegetation Index (NDVI)**
- **Enhanced Vegetation Index (EVI)**
- **Ratio Vegetation Index (RVI)**
- **Soil Adjusted Vegetation Index (SAVI)**. 

Puede encontrar más información sobre índices de vegetación 
- [Productos espectrales del ODC](https://www.opendatacube.org/dcal-spectral-products)
- [Base de datos de los índices de vegetación](https://www.indexdatabase.de/)

### Normalized Difference Vegetation Index (NDVI)

El  NDVI  es  empleado  para calificar  el  verdor  de  la vegetación  y  es  útil para  evaluar su densidad y salud. Para este índice los valores cercanos a `1` corresponden a una vegetación densa, como la  encontrada  en  bosques o  cultivos  en  su  etapa  de  crecimiento  máximo, mientras que  los valores  cercanos  a  `0` representan  zonas  cuya  vegetación  es  escasa. Finalmente, valores negativos cercanos a `-1` representan indicios de agua.

Una presentación clara y concisa del NVDI la encontramos [aquí](https://eos.com/es/blog/ndvi-preguntas-frecuentes/)

La ecuación que define este índice es:

`NDVI = (NIR - RED) / (NIR + RED)`

**Cálculo del NDVI**

El open data cube permite operar la información espectral de una imágen de forma sencilla, de manera que el cálculo del NDVI se reduce a replicar la formula mostrada anteriormente. 

Observe que el resultado es agregado al dataset como una variable de datos nueva. Es decir que el índice fue calculado sobre todas las imágenes contenidas en el dataset.

In [ ]:
dataset['ndvi'] = (dataset.nir - dataset.red) / (dataset.nir + dataset.red)
dataset

Y claro, queremos visualizar la imagen resultado del NDVI calculado. Lo hacemos empleando la función `plot` simple.

In [ ]:
dataset.ndvi.plot(col='time',col_wrap=4)

**NOTA:** 
La imágenes anteriores no parecen reflejar los resultados esperados. En primera instancia, la barra de colores muestra que el ndvi calculado varia entre `0` y `60000`, cuando, según la literatura, el cálculo del NDVI entrega valores que varían entre `-1.0` y `1.0`. Por otro lado, la imagen se torna de un único color.

Es necesario mirar los resultados del cálculo de NDVI con más detalle:

In [ ]:
dataset.ndvi.plot()

El histograma muestra que la gran mayoria de valores están entre 0 y 7000, aproximadamente, pero también que hay algunos valores del orden de 60000 !!. Esto nos lo confirman los datos numéricos, donde precisamente el último valor calculado es 65535!!

La explicación de esto no es simple, aunque es bien conocida:

Cuando se trata con imágenes satelitales es común encontrar valores de píxeles inválidos
- Pixeles con valores que están por fuera del rango válido de valores de las bandas
- Píxeles que no tienen información
- Píxeles que presentan nubes poco densas que no son visibles a simple vista o por el nivel de detalle de la imagen se hace imperceptible, entre otros casos. 

Dado lo anterior, es prudente y necesario remover estos píxeles del análisis original para evitar estos errores y su propagación en los análisis que siguen.

**Enmascaramiento de píxeles inválidos**

El enmascarado es el proceso de eliminar o remover información de píxeles no validos de la imágen para evitar propagar el error al hacer cálculos con estos valores. Una forma de enmascarar la imágen es usando los rangos de valores conocidos del NDVI como criterio de aceptación o eliminación de píxeles.

In [ ]:
# Generación de máscara que establece que deseamos dejar aquellos píxeles que presentan un ndvi mayor que -1.0
mask_lower = dataset.ndvi >= -1.0

# Generación de máscara que establece que deseamos dejar aquellos píxeles que son menores que 1.0
mask_higher = dataset.ndvi <= 1.0

# Aplicamos ambas máscaras sobre todo el dataset
masked_dataset = dataset.where(mask_lower & mask_higher)

# Imagen del NDVI después de haber removido los valores inválidos para el índice
masked_dataset.ndvi.plot(col='time',col_wrap=4)

**Definir diferentes colores para rangos establecidos de valores**

Para facilitar la publicación y análisis de resultados, es conveniente establecer colores específicos (definir una paleta de colores) para ciertos rangos de valores que permitan distinguir aspectos puntuales de la cobertura estudiada. El código a continuación establece colores específicos para rangos definidos de valores del NDVI.

Referencia: los colores empleados en al barra de colores fueron tomados de *[A repository of custom scripts that can be used with Sentinel-Hub services](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndvi/)*

In [ ]:
# Definición de colores para cada rango establecido en 'bounds'
cmap = mpl.colors.ListedColormap(
    [
        '#000000', 
        '#a50026',
        '#d73027',
        '#f46d43',
        '#fdae61',
        '#fee08b',
        '#ffffbf',
        '#d9ef8b',
        '#a6d96a',
        '#66bd63',
        '#1a9850',
        '#006837'
    ]
)

# Rangos de valores establecidos
bounds = [-1.0, -0.2, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Genera una capa de normalización de los datos basada en los intervalos establecidos en 'bounds'
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

# Mostrar la variable de datos NDVI, con la clasificación y la paleta de colores definida.
masked_dataset.ndvi.plot(col='time',col_wrap=4,cmap=cmap,norm=norm)

### Enhanced Vegetation Index (EVI)

El índice EVI es similar al NDVI; sin embargo, corrige algunas condiciones atmosféricas y es más sensible en áreas con alta densidad de vegetación. La ecuación que describe el cálculo de este índice se muestra a continuación:

`2 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 10000))`

> **TODO:**  Realice el mismo proceso de análisis del NDVI, pero en este caso para el cálculo del EVI. Utilice nuevas celdas para hacerlo

## 4. Guardar resultados de análisis en formato netcdf
___

In [ ]:
# Selecciono el periodo de tiempo que deseo guardar
time_index = 0
dataset_to_save = masked_dataset.isel(time=time_index)

# Selecciono la banda que deseo guardar
ndvi = dataset_to_save.ndvi

# Elimino la coordenada 'time' del dataset 
ndvi = ndvi.drop('time')
ndvi

Guardar resultados de análisis en un archivo .nc

In [ ]:
ndvi.to_netcdf('Salidas/ndvi.nc')

## 5. Guardar resultados de análisis en formato geotiff
___

Funciones requeridas para guardar información en geotiff

In [ ]:
"""
Las funciones mostradas a continuación fueron tomadas de 
https://github.com/ceos-seo/data_cube_notebooks
"""

def _get_transform_from_xr(data, x_coord='longitude', y_coord='latitude'):
    """Create a geotransform from an xarray.Dataset or xarray.DataArray.
    """

    from rasterio.transform import from_bounds
    geotransform = from_bounds(data[x_coord][0], data[y_coord][-1],
                               data[x_coord][-1], data[y_coord][0],
                               len(data[x_coord]), len(data[y_coord]))
    return geotransform

def write_geotiff_from_xr(tif_path, data, bands=None, no_data=-9999, crs="EPSG:4326",
                          x_coord='longitude', y_coord='latitude'):
    """
    NOTE: Instead of this function, please use `import_export.export_xarray_to_geotiff()`.
    Export a GeoTIFF from an `xarray.Dataset`.
    Parameters
    ----------
    tif_path: string
        The path to write the GeoTIFF file to. You should include the file extension.
    data: xarray.Dataset or xarray.DataArray
    bands: list of string
        The bands to write - in the order they should be written.
        Ignored if `data` is an `xarray.DataArray`.
    no_data: int
        The nodata value.
    crs: string
        The CRS of the output.
    x_coord, y_coord: string
        The string names of the x and y dimensions.
    """
    if isinstance(data, xr.DataArray):
        height, width = data.sizes[y_coord], data.sizes[x_coord]
        count, dtype = 1, data.dtype
    else:
        if bands is None:
            bands = list(data.data_vars.keys())
        else:
            assrt_msg_begin = "The `data` parameter is an `xarray.Dataset`. "
            assert isinstance(bands, list), assrt_msg_begin + "Bands must be a list of strings."
            assert len(bands) > 0 and isinstance(bands[0], str), assrt_msg_begin + "You must supply at least one band."
        height, width = data.dims[y_coord], data.dims[x_coord]
        count, dtype = len(bands), data[bands[0]].dtype
    with rasterio.open(
            tif_path,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=count,
            dtype=dtype,
            crs=crs,
            transform=_get_transform_from_xr(data, x_coord=x_coord, y_coord=y_coord),
            nodata=no_data) as dst:
        if isinstance(data, xr.DataArray):
            dst.write(data.values, 1)
        else:
            for index, band in enumerate(bands):
                dst.write(data[band].values, index + 1)
    dst.close()

Guardar resultados de análisis en un archivo .tif

In [ ]:
write_geotiff_from_xr(
    tif_path='Salidas/ndvi.tif', 
    data=ndvi, 
    bands=['ndvi'], 
    no_data=-9999, 
    crs="EPSG:4326",
    x_coord='longitude',
    y_coord='latitude'
)

**Ya puede visualizar y utilizar los resultados guardados en el SIG de su elección...**